## Import libraries

In [ ]:
!pip install PyPDF2
!pip install xlsxwriter
!pip install nltk

In [ ]:
import pandas as pd
from google.colab import drive
import os
import IPython
import PyPDF2
import shutil
from PyPDF2 import PdfReader
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Connect with drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Create folder

In [ ]:
# Define the folder path in your Google Drive
general_path= '/content/drive/My Drive/TrabajoWater/Archivos'

# Check if the folder exists, if not, create it
if not os.path.exists(general_path):
    os.makedirs(general_path)
    print("Folder was created")

## Extract text from pdf - Full paper

In [ ]:
# Function to extract text from PDF files
def extract_text_from_pdf(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file, strict=False)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [ ]:
def extract_REFERENCES_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        num_pages = len(reader.pages)

        # Initialize variables
        references_text = ""
        references_found = False

        # Loop through each page
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            page_text = page.extract_text()

            # Check if "REFERENCES" is in the page text
            if "REFERENCES" in page_text:
                references_found = True

                # Append the content starting from "REFERENCES" to the result
                references_index = page_text.find("REFERENCES")
                references_text += page_text[references_index:]

                # Append text from subsequent pages
                for next_page_num in range(page_num + 1, num_pages):
                    next_page = reader.pages[next_page_num]
                    next_page_text = next_page.extract_text()
                    references_text += next_page_text

                # No need to process further pages, break the loop
                break
        print(references_text.strip())
        return references_text.strip()

In [ ]:
# Array to store text from all files
texts = []
titles = []

# Folder with the pdf files
path_with_pdfs = os.path.join(general_path)

# Loop through each file in the folder
for filename in os.listdir(path_with_pdfs):
    file_path = os.path.join(path_with_pdfs, filename)



    try:
      # Check if the file is a PDF
      if filename.endswith('.pdf'):
          text = extract_text_from_pdf(file_path

          print(f"Title: {filename}")

          references_text = extract_REFERENCES_from_pdf(file_path)

          # Remove REFERENCES text from main text
          if references_text:
              print(references_text)
              text = text.replace(references_text, '')

          # Update the texts list with the modified text
          texts.append(text)
          titles.append(filename)
          print("="*50)
    except:
      continue

Title: The working for water programme_ Evolution of a payments for ecosystem services mechanism that addresses both poverty and ecosystem service delivery in South Africa.pdf


REFERENCES  
 
Ashton, P.J., 2002. Avoiding conflicts over Af rica’s water resources. Ambio 31, 236-242. 
Ashton, P.J., Haasbroek, B., 2002. Water de mand management and social adaptive capacity : a South African case study. In: Turton , 
A.R., Henwood, R. (Eds.), Hydropolitics in the Developing Worl d: A Southern African Persp ective. African Water Issues  
Research Unit (AWIRU) and International Wa ter Management Institute (IWMI). 24 pp.  
Ashton, P.J., Seetal, A., 2002. Challenges of water resource manage ment in Africa. African Renais -Science Conference, Durban, 1 2-
15 March. Baskin, Y., 1996. Curbing undesira ble invaders. BioScience 46, 732-737.  
Blignaut, J.N., De Wit, M.P. (Eds.), 2004. Sustainable options. UCT press, Cape Town. 
Blignaut, J.N., Marais, C, Turpie, J.K., 2007. Determining a charge  for invasive alien plant speci es (IAPs) to augment water su pply in 
South Africa. Water SA 33, 27-34.  
Cowling, R.M., Pressey, R.L., Rouget, M., Lombard, AT., 2003. A conservati

Title: Water Resources Research - 1997 - Espey - Price elasticity of residential demand for water  A meta‐analysis.pdf



Title: The widespread and unjust drinking water and clean water crisis in the United States.pdf

Title: Water poverty in England and Wales.pdf

Title: Water and beverage consumption among children age 4-13y in the United States_ analyses of 2005–2010 NHANES data.pdf

Title: Water Subsidy Policies_ A Comparison of the Chilean and Colombian Schemes.pdf
REFERENCES
Brook, P. J., and S. M. Smith. 2001. Contracting for Public Services: Output-Based Aid and Its
Applications . Washington, D.C.: World Bank.
Estache, A., V. Foster, and Q. Wodon. 2002. Accounting for Poverty in Infrastructure Reform: Learning
from Latin America’s Experience . WBI Development Studies. Washington D.C.: World Bank
Institute.
Estache, A., D. Leipziger, and A. Go ´mez-Lobo. 2001. ‘‘Utilities ‘Privatization’ and the Poor’s Needs in
Latin America: Have We Learned Enough to Get It Right?’’ World Development 29(7):1179–98.
Go´mez-Lobo, A., and D. Contreras. 2000. ‘‘Subsidy Policies for the Utility Industries: A Compariso

In [ ]:
df = pd.DataFrame({'FullText': texts,'Title': titles})

In [ ]:
df.head()

,FullText,Title
0,The working for water programme: evolution of ...,The working for water programme_ Evolution of ...
1,"WATER RESOURCES RESEARCH, VOL. 33, NO. 6, PAGE...",Water Resources Research - 1997 - Espey - Pric...
2,ARTICLE\nThe widespread and unjust drinking wa...,The widespread and unjust drinking water and c...
3,\n \nWATER POVERTY IN ENGLAND AND WALES \n ...,Water poverty in England and Wales.pdf
4,RESEARCH Open Access\nWater and beverage consu...,Water and beverage consumption among children ...
5,Water Subsidy Policies: A Comparison of the\nC...,Water Subsidy Policies_ A Comparison of the Ch...
6,"Water, Electricity, \nand the Poor\nWho Benefi...","Water, Electricity and the Poor_ Who Benefits ..."
7,ZK[KF ZIN FZ\OIS K\n_kto~ kqq{~nkltwtty kzn su...,Water affordability and human right to water i...
8,"Development and Change , Vol. 39, Issue 1, Jan...",Waiting for Miracles_ The Commercialization of...
9,sustainability\nArticle\nTrend Analysis of Wat...,Trend Analysis of Water Poverty Index for Asse...


In [ ]:
df.to_csv("/content/drive/My Drive/TrabajoWater/archivos.csv", escapechar="\\")
